# Shor's Algorithm
This implementation was authored by Gregory Croisdale <br>
5/1/2020, COSC 494, University of Tennesse, Knoxville

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def collapse(n,      # number we're trying to factor
             a,      # coprime with n
             rem=-1):# target remainder
    """Find states which yield certain remainder"""
    # initialize remainder and catch stop recursion case
    if rem == -1:
        rem = n - 1
    if rem == 0:
        raise ValueError("No valid states found")
    
    # find mod inverses
    end = 2 ** n.bit_length()
    out = []
    for i in range(end):
        if (a ** i) % n == rem:
            out += [i]
    
    # if unsuccessful, try reducing remainder
    if len(out) == 0:
        return collapse(n, a, rem=rem-1)
    
    return out

In [3]:
def quantum_ft(n, guess):
    """Performs a quantum fourier transformation so we can find periods. Large
       portions of this code are from the Qiskit reference material.           """
    from qiskit import QuantumCircuit, QuantumRegister, execute, Aer, IBMQ
    from numpy import pi
    
    # The following code is from qiskit reference:
    def qft_rotations(circuit, n):
        """Performs qft on the first n qubits in circuit (without swaps)"""
        if n == 0:
            return circuit
        n -= 1
        circuit.h(n)
        for qubit in range(n):
            circuit.cu1(pi/2**(n-qubit), qubit, n)
        # At the end of our function, we call the same function again on
        # the next qubits (we reduced n by one earlier in the function)
        qft_rotations(circuit, n)
    
    def swap_registers(circuit, n):
        for qubit in range(n//2):
            circuit.swap(qubit, n-qubit-1)
        return circuit

    def qft(circuit, n):
        """QFT on the first n qubits in circuit"""
        qft_rotations(circuit, n)
        swap_registers(circuit, n)
        return circuit
    
    # The following code is my own:
    # mark qubits with designated modulo value
    marked = collapse(n, guess)
    
    # create a democratic state from all marked qubits
    state = [1 / np.sqrt(len(marked)) if i in marked else 0 for i in range(2 ** n.bit_length())]
    q = QuantumRegister(n.bit_length())
    qc = QuantumCircuit(q)
    qc.initialize(state, q)
    
    # perform qft twice
    qft(qc,n.bit_length())
    qft(qc,n.bit_length())

    backend = Aer.get_backend("statevector_simulator")
    statevector = execute(qc, backend=backend).result().get_statevector()
    return np.abs(statevector) ** 2

In [4]:
def classical_ft(n, guess):
    """Performs a classical fourier transformation so we can find periods."""
    # mark "qubits" with designated modulo value
    marked = collapse(n, guess)
    state = [1 / np.sqrt(len(marked)) if i in marked else 0 for i in range(2 ** n.bit_length())]

    return np.fft.fft(np.fft.fft(np.abs(state) ** 2))

In [5]:
def find_period(ft_result):
    """Finds the period from a completed fourier transformation."""
    probs = np.abs((ft_result)) ** 2
    peaks = np.where(probs > np.amax(probs) / 2)[0]
    if len(peaks) < 2:
        return 1
    return peaks[1] - peaks[0]

In [6]:
def prime_from_period(a, # "guess" - coprime with n
                      r, # period
                      n):# number we're factoring
    """Turns a found period into two factors"""
    prime1 = np.gcd(a ** (r // 2) - 1, n)
    if prime1 == 1:
        prime1 = np.gcd(a ** (r // 2) + 1, n)
    prime2 = n // prime1
    assert prime1 * prime2 == n
    
    return prime1, prime2

In [11]:
def shor_factor(
    n,
    guess=3,
    mode="quantum_ft"):
    """Factors an integer using a designated method"""
    ft = None
    if (mode == "quantum_ft"):
        ft=quantum_ft     
    elif (mode == "classical_ft"):
        ft=classical_ft
    else:
        raise TypeError(mode)
        
    # keep guessing until you get a helpful period
    while (guess < n):
        print(guess)
        # perform qft and find period
        period = find_period(ft(n, guess))
        # turn period into two primes
        prime1, prime2 = prime_from_period(guess, period, n)
        # ignore n, 1 pairs
        if (abs(prime1 - prime2) != n - 1):
            return prime1, prime2

        # find a new guess
        guess += 1
        while np.gcd(guess, n) != 1 and guess < n:
            guess += 1
        
    return 1, n

In [13]:
from time import time

time_0 = time()
p1, p2 = shor_factor(7, mode="classical_ft")
time_1 = time()
print(p1, p2, time_1 - time_0)

time_0 = time()
p1, p2 = shor_factor(1035, mode="quantum_ft")
time_1 = time()
print(p1, p2, time_1 - time_0)

3
4
5
6
1 7 0.0012319087982177734
3
23 45 0.25469064712524414
